#### Initialize

In [ ]:
from pathlib import Path

import maples_dr
import numpy as np
import pandas as pd
from jppype import Mosaic, vscode_theme

from fundus_vessels_toolkit import AVLabel, FundusData
from fundus_vessels_toolkit.pipelines import FundusAVSegToBiomarkers
from fundus_vessels_toolkit.pipelines.avseg_to_tree import AVSegToTree, NaiveAVSegToTree
from fundus_vessels_toolkit.vparameters import parametrize_bifurcations

vscode_theme()

In [ ]:
maples_dr.configure(messidor_path="/home/gaby/These/Data/Fundus/MESSIDOR/", image_format="rgb")
dataset = maples_dr.load_dataset()

In [ ]:
GRAPH_AV_COLORS = {
    AVLabel.BKG: "grey",
    AVLabel.ART: "red",
    AVLabel.VEI: "blue",
    AVLabel.BOTH: "purple",
    AVLabel.UNK: "green",
}

In [ ]:
av2tree = AVSegToTree()
naive_av2tree = NaiveAVSegToTree()
av2biomarkers = AVSegToTree()

#### Load image

In [ ]:
IMG = Path("20060523_49681_0100_PP.png")  # GOOD
PRED_PATH = Path("/home/gaby/These/src/Fundus/RelatedWorks/AutoMorph/Results/M2/artery_vein/raw")
GT_PATH = Path("/home/gaby/These/Data/Fundus/MAPLES-DR/AV_reviewed/")

AV_PRED = PRED_PATH / IMG.name
sample = dataset[IMG.stem]
fundus = FundusData(
    fundus=sample["fundus"],
    fundus_mask=sample["mask"],
    od=sample["opticDisc"],
    vessels=AV_PRED,
    macula=sample["macula"],
)

av_gt = FundusData.load_vessels(GT_PATH / IMG.name)
v, a = av_gt == AVLabel.ART, av_gt == AVLabel.VEI
av_gt[a] = AVLabel.ART
av_gt[v] = AVLabel.VEI

fundus_gt = FundusData(
    fundus=sample["fundus"],
    fundus_mask=sample["mask"],
    od=sample["opticDisc"],
    vessels=av_gt,
    macula=sample["macula"],
)

## Extract Vascular Graph

In [ ]:
av_graph = av2tree.to_vgraph(fundus)

m = Mosaic(2, cols_titles=["Ground Truth", "IA Segmentation"])
fundus_gt.draw(view=m[0], labels_opacity=0.8)
fundus.draw(view=m[1], labels_opacity=0.8)


# m[1]["graph"] = av_graph.jppype_layer(bspline=True, node_labels=False)
# m[1]["tangents"] = av_graph.geometric_data().jppype_branches_tips_tangents(scaling=10)

m.show()

## Optimize Sub-Trees

In [ ]:
lines_digraph_info = av2tree.build_line_digraph(av_graph, fundus)
opti_tree = av2tree.resolve_digraph_to_vtree(*lines_digraph_info)
a_tree, v_tree = av2tree.split_av_tree(opti_tree)

In [ ]:
naive_a_tree, naive_v_tree = naive_av2tree(fundus)

In [ ]:
def colorize_tree(tree, art, layer):
    import webcolors

    if art:
        root_color = "#7a1a1a"
        color0 = np.array(webcolors.hex_to_rgb("#ff0000"))
        leaf_color = "#da7676"
        color_max = np.array(webcolors.hex_to_rgb("#ff7a7a"))
        label = AVLabel.ART
    else:
        root_color = "#1a1a7a"
        color0 = np.array(webcolors.hex_to_rgb("#0000ff"))
        leaf_color = "#7676da"
        color_max = np.array(webcolors.hex_to_rgb("#7a7aff"))
        label = AVLabel.VEI
    main_color = GRAPH_AV_COLORS[label]
    nodes_color = pd.Series(main_color, index=tree.nodes_attr.index)
    nodes_color[tree.root_nodes_ids()] = root_color
    nodes_color[tree.leaf_nodes_ids()] = leaf_color
    layer.nodes_cmap = nodes_color.to_dict()

    if "rank" in tree.nodes_attr:
        MAX_RANK = 4
        edge_gradient = np.linspace(color0, color_max, MAX_RANK, endpoint=True).astype(int)
        edges_rank = tree.nodes_attr["rank"][tree.branch_head()].clip(1, MAX_RANK) - 1
        layer.edges_cmap = [webcolors.rgb_to_hex(tuple(edge_gradient[x])) for x in edges_rank]
    else:
        layer.edges_cmap = main_color


def colorize_subtrees(tree, layer, color_shift=0):
    def colormap(x):
        cmap = ["red", "blue", "purple", "green", "orange", "cyan", "pink", "yellow", "teal", "lime"]
        return cmap[(x + color_shift) % len(cmap)]

    RANK_COLOR_MAP = {
        -1: "#f9ecdd",
        0: "#3a37ff",
        1: "#6f00ff",
        2: "#f36bd6",
        3: "#ff8282",
        4: "#ff9f79",
        5: "#ffcc92",
        6: "#ebe7b3",
        7: "#f9ecdd",
    }
    layer.edges_cmap = pd.Series(tree.subtrees_branch_labels()).map(colormap).to_dict()
    node_rank = tree.nodes_attr["rank"].map(RANK_COLOR_MAP).to_dict()
    for node in tree.leaf_nodes_ids():
        node_rank[node] = "white"
    for node in tree.root_nodes_ids():
        node_rank[node] = "black"
    layer.nodes_cmap = node_rank


parametrize_bifurcations(a_tree)
parametrize_bifurcations(v_tree)

m = Mosaic(3, cols_titles=["Naive Tree", "Optimized Tree", "Subtrees"])
fundus_gt.draw(view=m, labels_opacity=0.2)

m[0]["a_tree"] = naive_a_tree.jppype_layer(bspline=True)
m[0]["v_tree"] = naive_v_tree.jppype_layer(bspline=True)
colorize_tree(naive_a_tree, True, m[0]["a_tree"])
colorize_tree(naive_v_tree, False, m[0]["v_tree"])

m[1]["a_tree"] = a_tree.jppype_layer(bspline=True)
m[1]["v_tree"] = v_tree.jppype_layer(bspline=True)
colorize_tree(a_tree, True, m[1]["a_tree"])
colorize_tree(v_tree, False, m[1]["v_tree"])

m[2]["a_tree"] = a_tree.jppype_layer(bspline=True)
m[2]["v_tree"] = v_tree.jppype_layer(bspline=True)
colorize_subtrees(a_tree, m[2]["a_tree"])
colorize_subtrees(v_tree, m[2]["v_tree"], color_shift=len(a_tree.subtrees()))

m.show()

## Generate biomarkers

In [ ]:
av2biomarkers = FundusAVSegToBiomarkers(av2tree)

In [ ]:
widgets, trees, data = av2biomarkers.inspect_bifurcations(fundus, label_branches=True)
widgets

In [ ]:
widgets, trees, data = av2biomarkers.inspect_branches(fundus)
widgets